In [2]:
import os
import rootpath
import sys
sys.path.append(rootpath.detect())

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from analysis_utilities import PATH_TO_REPORT_REPO, PATH_TO_PROCESSED_DSAF_RESULTS, PATH_TO_PROCESSED_PAREGO_RESULTS
from analysis_utilities import PAREGO_FILES_LIST, DSAF_FILES_LIST
from analysis_utilities import load_result, save_table
from analysis_utilities import UNATTAINABLE_DSAF_FILE_LIST, UNATTAINABLE_PAREGO_FILE_LIST
from analysis_utilities import PARETO_DSAF_FILE_LIST, PARETO_PAREGO_FILE_LIST
from analysis_utilities import ATTAINABLE_DSAF_FILE_LIST, ATTAINABLE_PAREGO_FILE_LIST

In [5]:
def save_table(*args, **kwargs):
    pass
save_table(1, a=2)

In [6]:
TABLE_STAGES = [25, 50, 100, 150]

In [7]:
def get_med_iqr(file_name, file_dir, attainable=False):
    result = load_result(file_name, file_dir)
    stage_inds = [np.where(np.asarray(result.dual_hpv_hist_x)==ts)[0][0] for ts in TABLE_STAGES] 
    
    if attainable:
        att_ind = 1
    else:
        att_ind = 0
    med = np.median(result.dual_hpv_history[att_ind][:, stage_inds], axis=0)
    lqr = np.quantile(result.dual_hpv_history[att_ind][:, stage_inds], 0.25, axis=0)
    uqr = np.quantile(result.dual_hpv_history[att_ind][:, stage_inds], 0.75, axis=0)
    return med, uqr-lqr


def get_table_dict_from_list(file_list, file_dir, attainable=False):
    D = {}
    for file_name in file_list:
        # get experiment name for table key
        key = file_name[:-7]
        
        med, iqr= get_med_iqr(file_name, file_dir, attainable)
        data0 = np.hstack((med.reshape(-1,1), iqr.reshape(-1,1)))
        if key+"_1" not in D.keys(): 
            D[key+f"_1"] = np.hstack([di for di in data0])#+[di for di in data1])
        else:
            D[key+f"_2"] = np.hstack([di for di in data0])#+[di for di in data1])
#         D[key] = np.hstack([di for di in data0])# +[dj for dj in data1])
    return D

In [8]:
def get_med_iqr_combined(file_name, file_dir):
    result = load_result(file_name, file_dir)
    stage_inds = [np.where(np.asarray(result.dual_hpv_hist_x)==ts)[0][0] for ts in TABLE_STAGES] 
    
    dual_hpv_history = result.dual_hpv_history[0]+result.dual_hpv_history[1]
    med = np.median(dual_hpv_history[:, stage_inds], axis=0)
    lqr = np.quantile(dual_hpv_history[:, stage_inds], 0.25, axis=0)
    uqr = np.quantile(dual_hpv_history[:, stage_inds], 0.75, axis=0)
    return med, uqr-lqr


def get_table_dict_from_list_combined(file_list, file_dir):
    D = {}
    for file_name in file_list:
        # get experiment name for table key
        key = file_name[:-7]
        
        med, iqr= get_med_iqr_combined(file_name, file_dir)
        data0 = np.hstack((med.reshape(-1,1), iqr.reshape(-1,1)))
        if key+"_1" not in D.keys(): 
            D[key+f"_1"] = np.hstack([di for di in data0])#+[di for di in data1])
        else:
            D[key+f"_2"] = np.hstack([di for di in data0])#+[di for di in data1])
#         D[key] = np.hstack([di for di in data0])# +[dj for dj in data1])
    return D

In [9]:
def apply_f_to_mask(df, mask, f):
    best_inds =  [*zip(*np.where(mask == 1))] 
    for i, j in best_inds:
        df.iloc[i, j] = f(df.iloc[i, j])

def format_table(df):
    df_str = df.astype(str)
    # italicise IQR columns
    mask_italics = np.zeros_like(df_str.values, dtype=bool)
    mask_italics[:,1::2] = True
    f_italics = lambda cell_value: r"\textit{"+cell_value+"}"
    apply_f_to_mask(df_str, mask_italics, f_italics)
    
    # get boolean mask of winners/same results for each
    values_directed = df_str.values[:,:int(df_str.values.shape[1]/2)]
    values_undirected = df_str.values[:,int(df_str.values.shape[1]/2):]
    
    # highlight best
    directed_wins = np.repeat((values_directed>values_undirected)[:, ::2], 2, axis=1)
    undirected_wins = np.repeat((values_directed<values_undirected)[:, ::2], 2, axis=1)
    mask_best = np.hstack([directed_wins, undirected_wins])
    f_best = lambda cell_value: r"\best "+cell_value
    apply_f_to_mask(df_str, mask_best, f_best)
    
    # highlight draws
    draw = np.repeat((values_directed==values_undirected)[:, ::2], 2, axis=1)
    mask_draw = np.hstack([draw, draw])
    f_draw = lambda cell_value: r"\statsimilar "+cell_value
    apply_f_to_mask(df_str, mask_draw, f_draw)

    return df_str

In [11]:
D_parego = get_table_dict_from_list(PAREGO_FILES_LIST, PATH_TO_PROCESSED_PAREGO_RESULTS, False)
D_dsaf = get_table_dict_from_list(DSAF_FILES_LIST, PATH_TO_PROCESSED_DSAF_RESULTS, False)

D_table_va = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}
D_table_va

{'wfg1_2obj_3dim_1': array([0.546     , 0.032     , 0.54872934, 0.03589933, 0.54872934,
        0.03589933, 0.55094991, 0.03404885, 0.5854    , 0.0206    ,
        0.591     , 0.027     , 0.5988    , 0.0426    , 0.5988    ,
        0.0426    ]),
 'wfg1_2obj_3dim_2': array([0.8026, 0.0538, 0.8256, 0.0302, 0.843 , 0.0361, 0.8444, 0.0295,
        0.7748, 0.0464, 0.8108, 0.0426, 0.8314, 0.0336, 0.8456, 0.0292]),
 'wfg1_3obj_4dim_1': array([0.56776   , 0.13481   , 0.59784575, 0.13560614, 0.59784575,
        0.11783355, 0.61461234, 0.11526268, 0.60774   , 0.12038   ,
        0.649     , 0.0605    , 0.66026   , 0.05163   , 0.6634    ,
        0.03095   ]),
 'wfg1_3obj_4dim_2': array([0.57974, 0.12639, 0.65102, 0.11163, 0.68392, 0.10442, 0.68518,
        0.09455, 0.697  , 0.09944, 0.7224 , 0.08846, 0.73418, 0.06369,
        0.73864, 0.04745]),
 'wfg1_4obj_5dim_1': array([0.42868 , 0.08967 , 0.46859 , 0.077935, 0.4717  , 0.08591 ,
        0.4743  , 0.078915, 0.49249 , 0.07502 , 0.50959 , 0.0642

In [9]:
D_parego = get_table_dict_from_list(PAREGO_FILES_LIST, PATH_TO_PROCESSED_PAREGO_RESULTS, True)
D_dsaf = get_table_dict_from_list(DSAF_FILES_LIST, PATH_TO_PROCESSED_DSAF_RESULTS, True)

D_table_vb = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

In [10]:
D_parego = get_table_dict_from_list_combined(PAREGO_FILES_LIST, PATH_TO_PROCESSED_PAREGO_RESULTS)
D_dsaf = get_table_dict_from_list_combined(DSAF_FILES_LIST, PATH_TO_PROCESSED_DSAF_RESULTS)

D_table_combined = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

In [11]:
D_parego = get_table_dict_from_list(ATTAINABLE_PAREGO_FILE_LIST, PATH_TO_PROCESSED_PAREGO_RESULTS, False)
D_dsaf = get_table_dict_from_list(ATTAINABLE_DSAF_FILE_LIST, PATH_TO_PROCESSED_DSAF_RESULTS, False)

D_table_attainable = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

mic = pd.MultiIndex.from_product([["dsaf", "ParEgo"], [25, 50, 100, 150],['median', 'IQR']])
mir = pd.MultiIndex.from_product([[d[:-2].replace("_", " ") for d in D_table_attainable.keys()][::2], ["1", "2"]], names=[r"$f$", "Position"])
df_attainable = pd.DataFrame.from_dict(D_table_attainable, orient="index", columns=mic).round(3)
df_attainable.index = mir
df_attainable = format_table(df_attainable)
save_table(df_attainable, "attainable_parego_table_va")
df_attainable

dsaf                             \
                                       25                               
                                    median                        IQR   
$f$             Position                                                
wfg1 2obj 3dim  1                    0.586             \textit{0.053}   
                2              \best 0.803       \best \textit{0.054}   
wfg1 3obj 4dim  1                    0.595             \textit{0.121}   
                2                     0.58             \textit{0.126}   
wfg1 4obj 5dim  1                    0.464             \textit{0.104}   
                2                    0.485             \textit{0.079}   
wfg2 2obj 6dim  1              \best 0.951       \best \textit{0.033}   
                2                    0.839             \textit{0.141}   
wfg2 3obj 6dim  1                    0.803             \textit{0.164}   
                2                    0.861             \textit{0.097}   
wfg2 4obj 10dim 1                    0.656             \textit{0.116}   
                2                     0.92             \textit{0.042}   
wfg3 2obj 6dim  1              \best 0.999       \best \textit{0.008}   
                2                \best 1.0       \best \textit{0.005}   
wfg3 3obj 10dim 1              \best 0.953       \best \textit{0.049}   
                2              \best 0.972       \best \textit{0.028}   
wfg3 4obj 10dim 1              \best 0.949       \best \textit{0.042}   
                2              \best 0.968        \best \textit{0.03}   
wfg4 2obj 6dim  1                    0.919             \textit{0.178}   
                2         \statsimilar 1.0  \statsimilar \textit{0.0}   
wfg4 3obj 8dim  1                      0.9             \textit{0.165}   
                2                    0.911             \textit{0.149}   
wfg4 4obj 8dim  1                    0.683             \textit{0.121}   
                2                    0.843             \textit{0.131}   
wfg5 2obj 6dim  1              \best 0.988       \best \textit{0.054}   
                2                \best 1.0       \best \textit{0.011}   
wfg5 3obj 8dim  1                    0.871             \textit{0.184}   
                2               \best 0.95        \best \textit{0.06}   
wfg5 4obj 10dim 1                    0.583              \textit{0.11}   
                2                     0.77             \textit{0.116}   
wfg6 2obj 6dim  1              \best 0.971       \best \textit{0.061}   
                2               \best 0.98        \best \textit{0.06}   
wfg6 3obj 8dim  1                    0.886             \textit{0.142}   
                2                    0.959             \textit{0.029}   
wfg6 4obj 10dim 1                    0.603             \textit{0.114}   
                2                    0.757             \textit{0.132}   

                                                                         \
                                       50                                 
                                    median                          IQR   
$f$             Position                                                  
wfg1 2obj 3dim  1                    0.598                \textit{0.07}   
                2              \best 0.826          \best \textit{0.03}   
wfg1 3obj 4dim  1                      0.6                \textit{0.12}   
                2                    0.651               \textit{0.112}   
wfg1 4obj 5dim  1                    0.479               \textit{0.081}   
                2                    0.529               \textit{0.079}   
wfg2 2obj 6dim  1              \best 0.985         \best \textit{0.016}   
                2                    0.848               \textit{0.134}   
wfg2 3obj 6dim  1                    0.841               \textit{0.136}   
                2              \best 0.902         \best \textit{0.063}   
wfg2 4obj 10dim 1                    0.679               \textit{0.

In [35]:
D_parego = get_table_dict_from_list(ATTAINABLE_PAREGO_FILE_LIST, PATH_TO_PROCESSED_PAREGO_RESULTS, True)
D_dsaf = get_table_dict_from_list(ATTAINABLE_DSAF_FILE_LIST, PATH_TO_PROCESSED_DSAF_RESULTS, True)

D_table_attainable_vb = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

mic = pd.MultiIndex.from_product([["dsaf", "ParEgo"], [25, 50, 100, 150],['median', 'IQR']])
mir = pd.MultiIndex.from_product([[d[:-2].replace("_", " ") for d in D_table_attainable_vb.keys()][::2], ["1", "2"]], names=[r"$f$", "Position"])
df_attainable_vb = pd.DataFrame.from_dict(D_table_attainable_vb, orient="index", columns=mic).round(3)
df_attainable_vb.index = mir
df_attainable_vb = format_table(df_attainable_vb)
save_table(df_attainable_vb, "attainable_vb_parego_table_vb")
df_attainable_vb

dsaf                               \
                                       25                                 
                                    median                          IQR   
$f$             Position                                                  
wfg1 2obj 3dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg1 3obj 4dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg1 4obj 5dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 3obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 4obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg3 2obj 6dim  1         \statsimilar 0.0  \statsimilar \textit{0.025}   
                2         \statsimilar 0.0   \statsimilar \textit{0.06}   
wfg3 3obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg3 4obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg4 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2              \best 0.089         \best \textit{0.141}   
wfg4 3obj 8dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg4 4obj 8dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg5 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2              \best 0.105         \best \textit{0.288}   
wfg5 3obj 8dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg5 4obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg6 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg6 3obj 8dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg6 4obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   

                                                                         \
                                       50                                 
                                    median                          IQR   
$f$             Position                                                  
wfg1 2obj 3dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg1 3obj 4dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg1 4obj 5dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 3obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \te

In [13]:
D_parego = get_table_dict_from_list(UNATTAINABLE_PAREGO_FILE_LIST, PATH_TO_PROCESSED_PAREGO_RESULTS, False)
D_dsaf = get_table_dict_from_list(UNATTAINABLE_DSAF_FILE_LIST, PATH_TO_PROCESSED_DSAF_RESULTS, False)

D_table_unattainable = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

mic = pd.MultiIndex.from_product([["dsaf", "ParEgo"], [25, 50, 100, 150],['median', 'IQR']])
mir = pd.MultiIndex.from_product([[d[:-2].replace("_", " ") for d in D_table_unattainable.keys()][::2], ["1", "2"]], names=[r"$f$", "Position"])
df_unattainable = pd.DataFrame.from_dict(D_table_unattainable, orient="index", columns=mic).round(3)
df_unattainable.index = mir
df_unattainable = format_table(df_unattainable)
save_table(df_unattainable, "unattainable_parego_table")
df_unattainable

dsaf                               \
                                         25                                 
                                      median                          IQR   
$f$             Position                                                    
wfg1 2obj 3dim  1                      0.546               \textit{0.032}   
                2                      0.666               \textit{0.041}   
wfg1 3obj 4dim  1                      0.568               \textit{0.135}   
                2                      0.531                \textit{0.07}   
wfg1 4obj 5dim  1                      0.429                \textit{0.09}   
                2                      0.423               \textit{0.081}   
wfg2 2obj 6dim  1                \best 0.871          \best \textit{0.06}   
                2                      0.778               \textit{0.115}   
wfg2 3obj 6dim  1                      0.767               \textit{0.121}   
                2                      0.839               \textit{0.092}   
wfg2 4obj 10dim 1                      0.654               \textit{0.101}   
                2                \best 0.913         \best \textit{0.036}   
wfg3 2obj 6dim  1         \statsimilar 0.922  \statsimilar \textit{0.027}   
                2                \best 0.954         \best \textit{0.029}   
wfg3 3obj 10dim 1                \best 0.892         \best \textit{0.036}   
                2                \best 0.931         \best \textit{0.051}   
wfg3 4obj 10dim 1                \best 0.895         \best \textit{0.063}   
                2                      0.919               \textit{0.059}   
wfg4 2obj 6dim  1                      0.854               \textit{0.138}   
                2                \best 0.964          \best \textit{0.02}   
wfg4 3obj 8dim  1         \statsimilar 0.874  \statsimilar \textit{0.108}   
                2                       0.79               \textit{0.141}   
wfg4 4obj 8dim  1                      0.698                \textit{0.13}   
                2                      0.778               \textit{0.167}   
wfg5 2obj 6dim  1                       0.91               \textit{0.075}   
                2                \best 0.952         \best \textit{0.037}   
wfg5 3obj 8dim  1                      0.792               \textit{0.154}   
                2                \best 0.847           \best \textit{0.1}   
wfg5 4obj 10dim 1                \best 0.566         \best \textit{0.113}   
                2                \best 0.726         \best \textit{0.116}   
wfg6 2obj 6dim  1                \best 0.914         \best \textit{0.053}   
                2                \best 0.892         \best \textit{0.065}   
wfg6 3obj 8dim  1                      0.812               \textit{0.116}   
                2                \best 0.914         \best \textit{0.048}   
wfg6 4obj 10dim 1                      0.554               \textit{0.112}   
                2                      0.731               \textit{0.101}   

                                                                           \
                                         50                                 
                                      median                          IQR   
$f$             Position                                                    
wfg1 2obj 3dim  1                      0.549               \textit{0.036}   
                2                \best 0.713         \best \textit{0.056}   
wfg1 3obj 4dim  1                      0.598               \textit{0.136}   
                2                      0.553               \textit{0.089}   
wfg1 4obj 5dim  1                      0.469               \textit{0.078}   
                2                       0.48               \textit{0.081}   
wfg2 2obj 6dim  1                \best 0.923         \best \textit{0.058}   
                2                      0.799               \textit{0.102}   
wfg2 3obj 6dim  1                  

In [14]:
D_parego = get_table_dict_from_list(PARETO_PAREGO_FILE_LIST, PATH_TO_PROCESSED_PAREGO_RESULTS, False)
D_dsaf = get_table_dict_from_list(PARETO_DSAF_FILE_LIST, PATH_TO_PROCESSED_DSAF_RESULTS, False)

D_table_pareto = {k1: np.hstack([v1, v2]) for (k1, v1), (k2, v2) in zip(D_dsaf.items(), D_parego.items())}

mic = pd.MultiIndex.from_product([["dsaf", "ParEgo"], [25, 50, 100, 150],['median', 'IQR']])
mir = pd.MultiIndex.from_product([[d[:-2].replace("_", " ") for d in D_table_pareto.keys()][::2], ["1", "2"]], names=[r"$f$", "Position"])
df_pareto = pd.DataFrame.from_dict(D_table_pareto, orient="index", columns=mic).round(3)
df_pareto.index = mir
df_pareto = format_table(df_pareto)
save_table(df_pareto, "pareto_parego_table")
df_pareto

dsaf                        \
                                  25                          
                               median                   IQR   
$f$             Position                                      
wfg1 2obj 3dim  1               0.551        \textit{0.049}   
                2         \best 0.763   \best \textit{0.04}   
wfg1 3obj 4dim  1               0.584        \textit{0.095}   
                2               0.601        \textit{0.093}   
wfg1 4obj 5dim  1               0.447        \textit{0.096}   
                2               0.473        \textit{0.095}   
wfg2 2obj 6dim  1         \best 0.923   \best \textit{0.06}   
                2               0.797        \textit{0.109}   
wfg2 3obj 6dim  1               0.795        \textit{0.091}   
                2                0.85        \textit{0.131}   
wfg2 4obj 10dim 1               0.636        \textit{0.084}   
                2         \best 0.914  \best \textit{0.036}   
wfg3 2obj 6dim  1         \best 0.959  \best \textit{0.029}   
                2         \best 0.976  \best \textit{0.018}   
wfg3 3obj 10dim 1         \best 0.903  \best \textit{0.037}   
                2         \best 0.947  \best \textit{0.031}   
wfg3 4obj 10dim 1         \best 0.916  \best \textit{0.046}   
                2                0.92        \textit{0.046}   
wfg4 2obj 6dim  1               0.881        \textit{0.108}   
                2         \best 0.984  \best \textit{0.009}   
wfg4 3obj 8dim  1               0.853         \textit{0.14}   
                2               0.844        \textit{0.175}   
wfg4 4obj 8dim  1               0.671        \textit{0.266}   
                2                0.82        \textit{0.121}   
wfg5 2obj 6dim  1               0.941        \textit{0.063}   
                2         \best 0.976  \best \textit{0.021}   
wfg5 3obj 8dim  1               0.786        \textit{0.133}   
                2         \best 0.907   \best \textit{0.05}   
wfg5 4obj 10dim 1         \best 0.602  \best \textit{0.148}   
                2               0.756        \textit{0.141}   
wfg6 2obj 6dim  1         \best 0.931  \best \textit{0.068}   
                2         \best 0.908   \best \textit{0.08}   
wfg6 3obj 8dim  1               0.804        \textit{0.128}   
                2               0.917        \textit{0.063}   
wfg6 4obj 10dim 1                 0.6        \textit{0.117}   
                2               0.761        \textit{0.077}   

                                                                           \
                                         50                                 
                                      median                          IQR   
$f$             Position                                                    
wfg1 2obj 3dim  1                      0.561               \textit{0.056}   
                2                \best 0.784         \best \textit{0.031}   
wfg1 3obj 4dim  1                      0.619               \textit{0.096}   
                2                      0.644               \textit{0.081}   
wfg1 4obj 5dim  1                      0.473                 \textit{0.1}   
                2                      0.517               \textit{0.062}   
wfg2 2obj 6dim  1                \best 0.959         \best \textit{0.043}   
                2                      0.815               \textit{0.105}   
wfg2 3obj 6dim  1                      0.832               \textit{0.109}   
                2         \statsimilar 0.898  \statsimilar \textit{0.112}   
wfg2 4obj 10dim 1                      0.697               \textit{0.107}   
                2                \best 0.958         \best \textit{0.029}   
wfg3 2obj 6dim  1                \best 0.993         \best \textit{0.016}   
                2                \best 0.997         \best \textit{0.006}   
wfg3 3obj 10dim 1                 \best 0.97         \best \textit{0.032}   
                2                \best 0.982     

In [15]:
df_combined = pd.DataFrame.from_dict(D_table_combined, orient="index", columns=mic).round(3)
df_combined.index = mir
df_combined = format_table(df_combined)
df_combined

dsaf                               \
                                         25                                 
                                      median                          IQR   
$f$             Position                                                    
wfg1 2obj 3dim  1                      0.546               \textit{0.032}   
                2                \best 0.803         \best \textit{0.054}   
wfg1 3obj 4dim  1                      0.568               \textit{0.135}   
                2                       0.58               \textit{0.126}   
wfg1 4obj 5dim  1                      0.429                \textit{0.09}   
                2                      0.485               \textit{0.079}   
wfg2 2obj 6dim  1                \best 0.871          \best \textit{0.06}   
                2                      0.839               \textit{0.141}   
wfg2 3obj 6dim  1                      0.767               \textit{0.121}   
                2                      0.861               \textit{0.097}   
wfg2 4obj 10dim 1                      0.654               \textit{0.101}   
                2                       0.92               \textit{0.042}   
wfg3 2obj 6dim  1         \statsimilar 0.922  \statsimilar \textit{0.027}   
                2                  \best 1.0         \best \textit{0.065}   
wfg3 3obj 10dim 1                \best 0.892         \best \textit{0.036}   
                2                \best 0.972         \best \textit{0.028}   
wfg3 4obj 10dim 1                \best 0.895         \best \textit{0.063}   
                2                \best 0.968          \best \textit{0.03}   
wfg4 2obj 6dim  1                      0.854               \textit{0.138}   
                2                \best 1.089         \best \textit{0.141}   
wfg4 3obj 8dim  1         \statsimilar 0.874  \statsimilar \textit{0.108}   
                2                      0.911               \textit{0.149}   
wfg4 4obj 8dim  1                      0.698                \textit{0.13}   
                2                      0.843               \textit{0.131}   
wfg5 2obj 6dim  1                       0.91               \textit{0.075}   
                2                \best 1.105         \best \textit{0.298}   
wfg5 3obj 8dim  1                      0.792               \textit{0.154}   
                2                 \best 0.95          \best \textit{0.06}   
wfg5 4obj 10dim 1                \best 0.566         \best \textit{0.113}   
                2                       0.77               \textit{0.116}   
wfg6 2obj 6dim  1                \best 0.914         \best \textit{0.053}   
                2                 \best 0.98          \best \textit{0.06}   
wfg6 3obj 8dim  1                      0.812               \textit{0.116}   
                2                      0.959               \textit{0.029}   
wfg6 4obj 10dim 1                      0.554               \textit{0.112}   
                2                      0.757               \textit{0.132}   

                                                                           \
                                         50                                 
                                      median                          IQR   
$f$             Position                                                    
wfg1 2obj 3dim  1                      0.549               \textit{0.036}   
                2                \best 0.826          \best \textit{0.03}   
wfg1 3obj 4dim  1                      0.598               \textit{0.136}   
                2                      0.651               \textit{0.112}   
wfg1 4obj 5dim  1                      0.469               \textit{0.078}   
                2                      0.529               \textit{0.079}   
wfg2 2obj 6dim  1                \best 0.923         \best \textit{0.058}   
                2                      0.848               \textit{0.134}   
wfg2 3obj 6dim  1                  

In [33]:
arr = df_combined.values
arr.shape

a = arr[:, :8:2]
b = arr[:, 8::2]
c = a>b
dsaf_wins = np.sum(c)

d = a<b
parego_wins = np.sum(d)

e = a==b
draw = np.sum(e)

print("win: \t", dsaf_wins)
print("lose: \t", parego_wins)
print("draw: \t", draw)

win: 	 80
lose: 	 59
draw: 	 5


### RANKED SCORES

In [34]:
def get_comparison_matrices(resulta, resultb, intervals):
    assert isinstance(intervals, list), "intervals must be a list. For a single interval n supply [n]"
    assert all([i in resulta.dual_hpv_hist_x for i in intervals]), "Not all supplied intervals exist in results"
    assert all([i in resultb.dual_hpv_hist_x for i in intervals]), "Not all supplied intervals exist in results"
    
    indsa = [np.where(np.asarray(resulta.dual_hpv_hist_x)==i)[0][0] for i in intervals]
    indsb = [np.where(np.asarray(resultb.dual_hpv_hist_x)==i)[0][0] for i in intervals]
    ra_attainable = resulta.dual_hpv_history[0][:, indsa]
    ra_unattainable = resulta.dual_hpv_history[1][:, indsa]
    rb_attainable = resultb.dual_hpv_history[0][:, indsb]
    rb_unattainable = resultb.dual_hpv_history[1][:, indsb]
        
    ra = ra_attainable+ra_unattainable
    rb = rb_attainable+rb_unattainable
    
    Ma = (ra>rb)
    Mb = (rb>ra)
    M_draw = (ra==rb)
    return Ma, Mb, M_draw

def get_rank_table_from_list(f_names, result_list_a, result_list_b):
    D = {}
    for f_name, ra, rb in zip(f_names, result_list_a, result_list_b):
        Ma, Mb, M_draw = get_comparison_matrices(ra, rb, TABLE_STAGES)
        score_1 = Ma.astype(float)*1
        score_2 = M_draw.astype(float)*.5
        score = (score_1+score_2).mean(axis=0)
        D[f_name[:-5]] = score 
    return D

In [18]:
def add_gs_mask_to_df(df, mask):
    new_df = df.copy()
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            new_df.iloc[i, j] = r"\cellcolor[gray]{"+str(mask[i, j])+"} " +new_df.iloc[i, j]
    return new_df

def grayscale_df_mask(df):
    desired_max = 0.6
    scaled_values = df.values.copy()
    scaled_values[scaled_values<0.5] = 0.5
    
    scaled_values = (scaled_values-0.5)*2
    
    return (1-(scaled_values*desired_max)).round(3)

def format_rank_df(df):
    cols = df.columns
    new_cols = []
    for i in range(cols.shape[0]):
        new_cols.append(list(cols[i]))
        for j in range(np.shape(new_cols[i])[0]):
            if new_cols[i][j][0].lower() == "w":
                new_cols[i][j] = new_cols[i][j].upper() 
    
    df_new = pd.DataFrame(df.round(3).values, columns=pd.MultiIndex.from_tuples(new_cols), index=df.index)
#     df_new =df_new.rename(columns = {old: new for old, new in zip(cols, new_cols)}).round(3)
    
    # define win
    mask_win = (df_new.values>0.5)
    f_win = lambda value: "\\best "+value
    
    # define_grayscale
    gs_mask = grayscale_df_mask(df_new) 
    
    # define draw 
    mask_draw = (df_new.values==0.5)
    f_draw = lambda value: "\\statsimilar "+value
    
    # apply masks
    df_new = df_new.astype(str)
#     apply_f_to_mask(df_new, mask_win, f_win)
#     apply_f_to_mask(df_new, mask_draw, f_draw)
    return add_gs_mask_to_df(df_new, gs_mask)

In [19]:
dsaf_files = [load_result(fi, PATH_TO_PROCESSED_DSAF_RESULTS) for fi in DSAF_FILES_LIST]
parego_files = [load_result(fi, PATH_TO_PROCESSED_PAREGO_RESULTS) for fi in PAREGO_FILES_LIST]

for n1, n2 in zip(DSAF_FILES_LIST, PAREGO_FILES_LIST):
    assert n1 == n2
func_list = DSAF_FILES_LIST

In [20]:
pair = {"0": "0",
        "1": "1",
        "2": "2",
        "3": "0", 
        "4": "1", 
        "5": "2"}

f_names = [i.split("_")[0] for i in DSAF_FILES_LIST]
f_obj = [i.split("_")[1] for i in DSAF_FILES_LIST]
f_dim = [i.split("_")[2] for i in DSAF_FILES_LIST]
f_t = [i.split("_")[-1][:-5] for i in DSAF_FILES_LIST]
f_attain = [pair[i] for i in f_t]

D = get_rank_table_from_list(func_list, dsaf_files, parego_files)

df_rank_raw = pd.DataFrame.from_dict(D)
df_rank_raw.index = ["25", "50", "100", "150"]
df_rank_raw.columns = pd.MultiIndex.from_tuples(zip(f_names, f_obj, f_dim, f_t, f_attain))
df_rank_raw.columns.names = ["f_names", "f_obj", "f_dim", "f_t", "f_attain"]
df_rank_raw

f_names       wfg1                                                    \
f_obj         2obj                                                     
f_dim         3dim                                                     
f_t              0         1         2         3         4         5   
f_attain         0         1         2         0         1         2   
25        0.129032  0.129032  0.193548  0.387097  0.677419  0.838710   
50        0.161290  0.096774  0.258065  0.451613  0.612903  0.580645   
100       0.129032  0.096774  0.258065  0.354839  0.451613  0.612903   
150       0.129032  0.129032  0.258065  0.322581  0.419355  0.516129   

f_names                                           ...      wfg6            \
f_obj         3obj                                ...      3obj             
f_dim         4dim                                ...      8dim             
f_t              0         1         2         3  ...         2         3   
f_attain         0         1         2         0  ...         2         0   
25        0.419355  0.290323  0.419355  0.370968  ...  0.354839  0.645161   
50        0.451613  0.290323  0.322581  0.258065  ...  0.774194  0.774194   
100       0.387097  0.290323  0.354839  0.258065  ...  0.903226  0.935484   
150       0.419355  0.290323  0.322581  0.225806  ...  1.000000  1.000000   

f_names                                                               \
f_obj                             4obj                                 
f_dim                            10dim                                 
f_t              4         5         0         1         2         3   
f_attain         1         2         0         1         2         0   
25        0.548387  0.419355  0.258065  0.322581  0.225806  0.516129   
50        0.774194  0.838710  0.193548  0.225806  0.322581  0.451613   
100       0.903226  1.000000  0.870968  0.548387  0.741935  0.870968   
150       0.967742  1.000000  0.935484  0.741935  0.870968  0.967742   

f_names                       
f_obj                         
f_dim                         
f_t              4         5  
f_attain         1         2  
25        0.354839  0.354839  
50        0.741935  0.709677  
100       0.870968  0.903226  
150       0.967742  0.935484  

[4 rows x 108 columns]

In [21]:
print("win %: ", np.mean(df_rank_raw.values))

win %:  0.5693324372759856


In [22]:
df_rank_overall = df_rank_raw.groupby(level=[0, 1, 2, 3], axis=1).mean()
df_rank_overall

f_names      wfg1                                                              \
f_obj        2obj                                                        3obj   
f_dim        3dim                                                        4dim   
f_t             0         1         2         3         4         5         0   
25       0.129032  0.129032  0.193548  0.387097  0.677419  0.838710  0.419355   
50       0.161290  0.096774  0.258065  0.451613  0.612903  0.580645  0.451613   
100      0.129032  0.096774  0.258065  0.354839  0.451613  0.612903  0.387097   
150      0.129032  0.129032  0.258065  0.322581  0.419355  0.516129  0.419355   

f_names                                ...      wfg6                      \
f_obj                                  ...      3obj                       
f_dim                                  ...      8dim                       
f_t             1         2         3  ...         2         3         4   
25       0.290323  0.419355  0.370968  ...  0.354839  0.645161  0.548387   
50       0.290323  0.322581  0.258065  ...  0.774194  0.774194  0.774194   
100      0.290323  0.354839  0.258065  ...  0.903226  0.935484  0.903226   
150      0.290323  0.322581  0.225806  ...  1.000000  1.000000  0.967742   

f_names                                                                        
f_obj                  4obj                                                    
f_dim                 10dim                                                    
f_t             5         0         1         2         3         4         5  
25       0.419355  0.258065  0.322581  0.225806  0.516129  0.354839  0.354839  
50       0.838710  0.193548  0.225806  0.322581  0.451613  0.741935  0.709677  
100      1.000000  0.870968  0.548387  0.741935  0.870968  0.870968  0.903226  
150      1.000000  0.935484  0.741935  0.870968  0.967742  0.967742  0.935484  

[4 rows x 108 columns]

In [23]:
df_rank_unattainable = df_rank_overall.iloc[:, ::3] 
df_rank_pareto = df_rank_overall.iloc[:, 1::3] 
df_rank_attainable = df_rank_overall.iloc[:, 2::3] 
df_rank_attainable

f_names      wfg1                                                        wfg2  \
f_obj        2obj                3obj                4obj                2obj   
f_dim        3dim                4dim                5dim                6dim   
f_t             2         5         2         5         2         5         2   
25       0.193548  0.838710  0.419355  0.258065  0.225806  0.387097  0.516129   
50       0.258065  0.580645  0.322581  0.290323  0.258065  0.354839  0.838710   
100      0.258065  0.612903  0.354839  0.225806  0.225806  0.322581  1.000000   
150      0.258065  0.516129  0.322581  0.193548  0.225806  0.322581  1.000000   

f_names                                ...      wfg5                      \
f_obj                  3obj            ...      3obj                4obj   
f_dim                  6dim            ...      8dim               10dim   
f_t             5         2         5  ...         2         5         2   
25       0.290323  0.387097  0.483871  ...  0.516129  0.483871  0.451613   
50       0.193548  0.225806  0.419355  ...  0.612903  0.677419  0.354839   
100      0.096774  0.129032  0.419355  ...  0.645161  0.806452  0.580645   
150      0.193548  0.096774  0.516129  ...  0.838710  0.838710  0.580645   

f_names                wfg6                                                    
f_obj                  2obj                3obj                4obj            
f_dim                  6dim                8dim               10dim            
f_t             5         2         5         2         5         2         5  
25       0.483871  0.612903  0.580645  0.354839  0.419355  0.225806  0.354839  
50       0.677419  0.838710  0.774194  0.774194  0.838710  0.322581  0.709677  
100      0.806452  0.935484  0.838710  0.903226  1.000000  0.741935  0.903226  
150      0.741935  0.935484  0.806452  1.000000  1.000000  0.870968  0.935484  

[4 rows x 36 columns]

In [24]:
df_wfg_2obj = df_rank_overall.iloc[:, df_rank_overall.columns.get_level_values(1)=='2obj'].groupby(level=[0, 1], axis=1).mean()
df_wfg_3obj = df_rank_overall.iloc[:, df_rank_overall.columns.get_level_values(1)=='3obj'].groupby(level=[0, 1], axis=1).mean()
df_wfg_4obj = df_rank_overall.iloc[:, df_rank_overall.columns.get_level_values(1)=='4obj'].groupby(level=[0, 1], axis=1).mean()

In [25]:
# totals by objective
df_wfg_totals = pd.concat([df_wfg_2obj.mean(axis=1), df_wfg_3obj.mean(axis=1), df_wfg_4obj.mean(axis=1)], axis=1)
df_wfg_totals.columns = ["\nobj = 2", "\nobj = 3", "\nobj = 4"]

df_wfg_totals["All"] = df_wfg_totals.mean(axis=1)
df_wfg_totals

,\nobj = 2,\nobj = 3,\nobj = 4,All
25,0.538082,0.470878,0.431900,0.480287
50,0.613799,0.538530,0.495520,0.549283
100,0.629032,0.611559,0.601254,0.613949
150,0.625000,0.637545,0.638889,0.633811


In [26]:
# totals by attainability 
df_wfg_attainablility = pd.DataFrame(index=df_wfg_4obj.index)
df_wfg_attainablility["$A$"] = df_rank_unattainable.mean(axis=1).values
df_wfg_attainablility["$P$"] = df_rank_pareto.mean(axis=1).values
df_wfg_attainablility["$U$"] = df_rank_attainable.mean(axis=1).values

# df_wfg_attainablility = format_rank_df(df_wfg_attainablility) 
df_wfg_attainablility

,$A$,$P$,$U$
25,0.475806,0.479391,0.485663
50,0.530466,0.552867,0.564516
100,0.597670,0.624104,0.620072
150,0.619176,0.642473,0.639785


In [27]:
# combined M = 2 & 3
df_wfg_23obj = pd.concat([df_wfg_2obj, df_wfg_3obj], axis=1)

df_wfg_23obj.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 2"]*6)+(["\\nobj = 3"]*6), [i[0] for i in df_wfg_23obj.columns])))
df_wfg_23obj.index = ["25", "50", "100", "150"] 
# df_wfg_23obj.index.names= ["\\optstep"]
df_wfg_23obj = format_rank_df(df_wfg_23obj)
df_wfg_23obj.columns.names= ["", "\optstep"]
save_table(df_wfg_23obj, "summary_table_parego_01_2obj3obj")
df_wfg_23obj

\nobj = 2                                 \
\optstep                         WFG1                           WFG2   
25        \cellcolor[gray]{1.0} 0.392    \cellcolor[gray]{1.0} 0.414   
50         \cellcolor[gray]{1.0} 0.36    \cellcolor[gray]{1.0} 0.441   
100       \cellcolor[gray]{1.0} 0.317  \cellcolor[gray]{0.954} 0.538   
150       \cellcolor[gray]{1.0} 0.296  \cellcolor[gray]{0.891} 0.591   

                                                                        \
\optstep                           WFG3                           WFG4   
25        \cellcolor[gray]{0.713} 0.739    \cellcolor[gray]{1.0} 0.495   
50          \cellcolor[gray]{0.6} 0.833  \cellcolor[gray]{0.903} 0.581   
100       \cellcolor[gray]{0.549} 0.876  \cellcolor[gray]{0.958} 0.535   
150       \cellcolor[gray]{0.471} 0.941    \cellcolor[gray]{1.0} 0.462   

                                                                        \
\optstep                           WFG5                           WFG6   
25        \cellcolor[gray]{0.962} 0.532  \cellcolor[gray]{0.813} 0.656   
50        \cellcolor[gray]{0.864} 0.613  \cellcolor[gray]{0.574} 0.855   
100       \cellcolor[gray]{0.816} 0.653  \cellcolor[gray]{0.574} 0.855   
150       \cellcolor[gray]{0.842} 0.632  \cellcolor[gray]{0.606} 0.828   

                            \nobj = 3                               \
\optstep                         WFG1                         WFG2   
25        \cellcolor[gray]{1.0} 0.347  \cellcolor[gray]{1.0} 0.446   
50        \cellcolor[gray]{1.0} 0.323  \cellcolor[gray]{1.0} 0.339   
100       \cellcolor[gray]{1.0} 0.301  \cellcolor[gray]{1.0} 0.306   
150       \cellcolor[gray]{1.0} 0.285  \cellcolor[gray]{1.0} 0.323   

                                                                      \
\optstep                           WFG3                         WFG4   
25        \cellcolor[gray]{0.761} 0.699  \cellcolor[gray]{1.0} 0.425   
50        \cellcolor[gray]{0.633} 0.806  \cellcolor[gray]{1.0} 0.425   
100       \cellcolor[gray]{0.509} 0.909  \cellcolor[gray]{1.0} 0.427   
150       \cellcolor[gray]{0.522} 0.898  \cellcolor[gray]{1.0} 0.495   

                                                                        
\optstep                           WFG5                           WFG6  
25          \cellcolor[gray]{1.0} 0.441    \cellcolor[gray]{1.0} 0.468  
50        \cellcolor[gray]{0.845} 0.629   \cellcolor[gray]{0.748} 0.71  
100        \cellcolor[gray]{0.62} 0.817  \cellcolor[gray]{0.509} 0.909  
150       \cellcolor[gray]{0.584} 0.847  \cellcolor[gray]{0.426} 0.978

In [28]:
# combined M = 2 & 3: split 1
df_wfg_23obj_1 = pd.concat([df_wfg_2obj], axis=1)
df_wfg_23obj_1.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 2"]*6), [i[0] for i in df_wfg_23obj_1.columns])))
df_wfg_23obj_1.index = ["25", "50", "100", "150"] 
# df_wfg_23obj_1.index.names= ["\\optstep"]
df_wfg_23obj_1 = format_rank_df(df_wfg_23obj_1)
df_wfg_23obj_1.columns.names= ["", "$\optstep$"]
save_table(df_wfg_23obj_1, "summary_table_parego_01_2obj3obj_1")
df_wfg_23obj_1

\nobj = 2                                 \
$\optstep$                         WFG1                           WFG2   
25          \cellcolor[gray]{1.0} 0.392    \cellcolor[gray]{1.0} 0.414   
50           \cellcolor[gray]{1.0} 0.36    \cellcolor[gray]{1.0} 0.441   
100         \cellcolor[gray]{1.0} 0.317  \cellcolor[gray]{0.954} 0.538   
150         \cellcolor[gray]{1.0} 0.296  \cellcolor[gray]{0.891} 0.591   

                                                                          \
$\optstep$                           WFG3                           WFG4   
25          \cellcolor[gray]{0.713} 0.739    \cellcolor[gray]{1.0} 0.495   
50            \cellcolor[gray]{0.6} 0.833  \cellcolor[gray]{0.903} 0.581   
100         \cellcolor[gray]{0.549} 0.876  \cellcolor[gray]{0.958} 0.535   
150         \cellcolor[gray]{0.471} 0.941    \cellcolor[gray]{1.0} 0.462   

                                                                          
$\optstep$                           WFG5                           WFG6  
25          \cellcolor[gray]{0.962} 0.532  \cellcolor[gray]{0.813} 0.656  
50          \cellcolor[gray]{0.864} 0.613  \cellcolor[gray]{0.574} 0.855  
100         \cellcolor[gray]{0.816} 0.653  \cellcolor[gray]{0.574} 0.855  
150         \cellcolor[gray]{0.842} 0.632  \cellcolor[gray]{0.606} 0.828

In [29]:
# combined M = 2 & 3: split 2
df_wfg_23obj_2 = pd.concat([df_wfg_3obj], axis=1)
df_wfg_23obj_2.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 3"]*6), [i[0] for i in df_wfg_23obj_2.columns])))
df_wfg_23obj_2.index = ["25", "50", "100", "150"] 
# df_wfg_23obj_2.index.names= ["\\optstep"]
df_wfg_23obj_2 = format_rank_df(df_wfg_23obj_2) 
df_wfg_23obj_2.columns.names= ["", "$\optstep$"]
save_table(df_wfg_23obj_2, "summary_table_parego_01_2obj3obj_2", index=False)
df_wfg_23obj_2

\nobj = 3                               \
$\optstep$                         WFG1                         WFG2   
25          \cellcolor[gray]{1.0} 0.347  \cellcolor[gray]{1.0} 0.446   
50          \cellcolor[gray]{1.0} 0.323  \cellcolor[gray]{1.0} 0.339   
100         \cellcolor[gray]{1.0} 0.301  \cellcolor[gray]{1.0} 0.306   
150         \cellcolor[gray]{1.0} 0.285  \cellcolor[gray]{1.0} 0.323   

                                                                        \
$\optstep$                           WFG3                         WFG4   
25          \cellcolor[gray]{0.761} 0.699  \cellcolor[gray]{1.0} 0.425   
50          \cellcolor[gray]{0.633} 0.806  \cellcolor[gray]{1.0} 0.425   
100         \cellcolor[gray]{0.509} 0.909  \cellcolor[gray]{1.0} 0.427   
150         \cellcolor[gray]{0.522} 0.898  \cellcolor[gray]{1.0} 0.495   

                                                                          
$\optstep$                           WFG5                           WFG6  
25            \cellcolor[gray]{1.0} 0.441    \cellcolor[gray]{1.0} 0.468  
50          \cellcolor[gray]{0.845} 0.629   \cellcolor[gray]{0.748} 0.71  
100          \cellcolor[gray]{0.62} 0.817  \cellcolor[gray]{0.509} 0.909  
150         \cellcolor[gray]{0.584} 0.847  \cellcolor[gray]{0.426} 0.978

In [30]:
df_wfg_4Tobj = pd.concat([df_wfg_4obj, df_wfg_attainablility, df_wfg_totals], axis=1)
df_wfg_4Tobj.index = ["25", "50", "100", "150"] 
df_wfg_4Tobj.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 4"]*6)+(["Overall"]*7), [i[0] if isinstance(i, tuple) else i for i in df_wfg_4Tobj.columns ])))
# df_wfg_23obj.index.names= ["\\optstep"]
df_wfg_4Tobj = format_rank_df(df_wfg_4Tobj) 
df_wfg_4Tobj.columns.names= ["", "$\optstep$"]

save_table(df_wfg_4Tobj, "summary_table_parego_02_4objTotal")
df_wfg_4Tobj

\nobj = 4                                 \
$\optstep$                         WFG1                           WFG2   
25          \cellcolor[gray]{1.0} 0.301  \cellcolor[gray]{0.942} 0.548   
50          \cellcolor[gray]{1.0} 0.274  \cellcolor[gray]{0.954} 0.538   
100         \cellcolor[gray]{1.0} 0.247  \cellcolor[gray]{0.884} 0.597   
150         \cellcolor[gray]{1.0} 0.242    \cellcolor[gray]{0.8} 0.667   

                                                                        \
$\optstep$                           WFG3                         WFG4   
25          \cellcolor[gray]{0.903} 0.581  \cellcolor[gray]{1.0} 0.349   
50           \cellcolor[gray]{0.664} 0.78  \cellcolor[gray]{1.0} 0.403   
100         \cellcolor[gray]{0.549} 0.876  \cellcolor[gray]{1.0} 0.409   
150         \cellcolor[gray]{0.522} 0.898  \cellcolor[gray]{1.0} 0.441   

                                                                          \
$\optstep$                           WFG5                           WFG6   
25            \cellcolor[gray]{1.0} 0.473    \cellcolor[gray]{1.0} 0.339   
50          \cellcolor[gray]{0.954} 0.538    \cellcolor[gray]{1.0} 0.441   
100         \cellcolor[gray]{0.788} 0.677  \cellcolor[gray]{0.639} 0.801   
150          \cellcolor[gray]{0.78} 0.683  \cellcolor[gray]{0.516} 0.903   

                                  Overall                                 \
$\optstep$                            $A$                            $P$   
25            \cellcolor[gray]{1.0} 0.476    \cellcolor[gray]{1.0} 0.479   
50           \cellcolor[gray]{0.964} 0.53  \cellcolor[gray]{0.936} 0.553   
100         \cellcolor[gray]{0.882} 0.598  \cellcolor[gray]{0.851} 0.624   
150         \cellcolor[gray]{0.857} 0.619   \cellcolor[gray]{0.83} 0.642   

                                                                          \
$\optstep$                            $U$                      \nobj = 2   
25            \cellcolor[gray]{1.0} 0.486  \cellcolor[gray]{0.954} 0.538   
50          \cellcolor[gray]{0.922} 0.565  \cellcolor[gray]{0.863} 0.614   
100          \cellcolor[gray]{0.856} 0.62  \cellcolor[gray]{0.845} 0.629   
150          \cellcolor[gray]{0.832} 0.64   \cellcolor[gray]{0.85} 0.625   

                                                                          \
$\optstep$                      \nobj = 3                      \nobj = 4   
25            \cellcolor[gray]{1.0} 0.471    \cellcolor[gray]{1.0} 0.432   
50          \cellcolor[gray]{0.953} 0.539    \cellcolor[gray]{1.0} 0.496   
100         \cellcolor[gray]{0.866} 0.612  \cellcolor[gray]{0.879} 0.601   
150         \cellcolor[gray]{0.834} 0.638  \cellcolor[gray]{0.833} 0.639   

                                           
$\optstep$                            All  
25             \cellcolor[gray]{1.0} 0.48  
50          \cellcolor[gray]{0.941} 0.549  
100         \cellcolor[gray]{0.863} 0.614  
150         \cellcolor[gray]{0.839} 0.634

In [31]:
df_wfg_4Tobj_1 = pd.concat([df_wfg_4obj], axis=1)
df_wfg_4Tobj_1.index = ["25", "50", "100", "150"] 
df_wfg_4Tobj_1.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 4"]*6)+(["Overall"]*7), [i[0] for i in df_wfg_4Tobj_1.columns])))
# df_wfg_23obj.index.names= ["\\optstep"]
df_wfg_4Tobj_1 = format_rank_df(df_wfg_4Tobj_1)
df_wfg_4Tobj_1.columns.names= ["", "$\optstep$"]

save_table(df_wfg_4Tobj_1, "summary_table_parego_02_4objTotal_1")
df_wfg_4Tobj_1

\nobj = 4                                 \
$\optstep$                         WFG1                           WFG2   
25          \cellcolor[gray]{1.0} 0.301  \cellcolor[gray]{0.942} 0.548   
50          \cellcolor[gray]{1.0} 0.274  \cellcolor[gray]{0.954} 0.538   
100         \cellcolor[gray]{1.0} 0.247  \cellcolor[gray]{0.884} 0.597   
150         \cellcolor[gray]{1.0} 0.242    \cellcolor[gray]{0.8} 0.667   

                                                                        \
$\optstep$                           WFG3                         WFG4   
25          \cellcolor[gray]{0.903} 0.581  \cellcolor[gray]{1.0} 0.349   
50           \cellcolor[gray]{0.664} 0.78  \cellcolor[gray]{1.0} 0.403   
100         \cellcolor[gray]{0.549} 0.876  \cellcolor[gray]{1.0} 0.409   
150         \cellcolor[gray]{0.522} 0.898  \cellcolor[gray]{1.0} 0.441   

                                                                          
$\optstep$                           WFG5                           WFG6  
25            \cellcolor[gray]{1.0} 0.473    \cellcolor[gray]{1.0} 0.339  
50          \cellcolor[gray]{0.954} 0.538    \cellcolor[gray]{1.0} 0.441  
100         \cellcolor[gray]{0.788} 0.677  \cellcolor[gray]{0.639} 0.801  
150          \cellcolor[gray]{0.78} 0.683  \cellcolor[gray]{0.516} 0.903

In [32]:
df_wfg_4Tobj_2 = pd.concat([df_wfg_attainablility, df_wfg_totals], axis=1)
df_wfg_4Tobj_2.index = ["25", "50", "100", "150"] 
df_wfg_4Tobj_2.columns = pd.MultiIndex.from_tuples(list(zip((["Overall"]*7), df_wfg_4Tobj_2.columns)))
# df_wfg_23obj.index.names= ["\\optstep"]
df_wfg_4Tobj_2 = format_rank_df(df_wfg_4Tobj_2)
df_wfg_4Tobj_2.columns.names= ["", "$\optstep$"]
# df_wfg_4Tobj_2.columns.names= ["", ""]


save_table(df_wfg_4Tobj_2, "summary_table_parego_02_4objTotal_2", index=False)
df_wfg_4Tobj_2

Overall                                 \
$\optstep$                            $A$                            $P$   
25            \cellcolor[gray]{1.0} 0.476    \cellcolor[gray]{1.0} 0.479   
50           \cellcolor[gray]{0.964} 0.53  \cellcolor[gray]{0.936} 0.553   
100         \cellcolor[gray]{0.882} 0.598  \cellcolor[gray]{0.851} 0.624   
150         \cellcolor[gray]{0.857} 0.619   \cellcolor[gray]{0.83} 0.642   

                                                                          \
$\optstep$                            $U$                      \nobj = 2   
25            \cellcolor[gray]{1.0} 0.486  \cellcolor[gray]{0.954} 0.538   
50          \cellcolor[gray]{0.922} 0.565  \cellcolor[gray]{0.863} 0.614   
100          \cellcolor[gray]{0.856} 0.62  \cellcolor[gray]{0.845} 0.629   
150          \cellcolor[gray]{0.832} 0.64   \cellcolor[gray]{0.85} 0.625   

                                                                          \
$\optstep$                      \nobj = 3                      \nobj = 4   
25            \cellcolor[gray]{1.0} 0.471    \cellcolor[gray]{1.0} 0.432   
50          \cellcolor[gray]{0.953} 0.539    \cellcolor[gray]{1.0} 0.496   
100         \cellcolor[gray]{0.866} 0.612  \cellcolor[gray]{0.879} 0.601   
150         \cellcolor[gray]{0.834} 0.638  \cellcolor[gray]{0.833} 0.639   

                                           
$\optstep$                            All  
25             \cellcolor[gray]{1.0} 0.48  
50          \cellcolor[gray]{0.941} 0.549  
100         \cellcolor[gray]{0.863} 0.614  
150         \cellcolor[gray]{0.839} 0.634